In [1]:
from selenium import webdriver
from src.common.utils import *
#import logging
#import logging.config

config = loadConfiguration('config/config.yaml')
#os.makedirs(os.path.dirname(str(config['logs']['handlers']['file']['filename'])), exist_ok=True)
#logging.config.dictConfig(config['logs'])
#logger = logging.getLogger(__name__)
logger = getLogger(__name__)
logger.debug("something")
logger.warning("test")

AttributeError: module 'logging' has no attribute 'configuration'